## 行人检测

前面我们介绍了：

1. 如何定位图片中目标的位置（滑动窗口+图像金字塔）
2. 如何从图片中提取出目标的特征（方向梯度直方图）
3. 如何进行分类（分类算法）

现在我们将通过这节实验完成一个行人检测的项目。
本节实验将会应用到前面学习的所有内容，来构建一个传统的目标检测算法。

### 基本流程

简单来说行人检测就是在提供的图像中，我们想要计算机分辨出哪些是人并且用矩形框标记出人出现在图片中的哪些位置。
下图左上角图片中有一个人，如果我们想要用传统的目标检测方法检测到这个人的话，一般分为下面几个步骤。

1. 使用图像金字塔将图片按一定缩放比例生成不同尺寸图片（下图序号 1 所示）。
2. 使用滑动窗口在每张不同尺寸的图片上从左至右、从上向下滑动（下图序号 2 所示）--> 这样会生成很多小的图片。
3. 将滑动窗口滑过的每个区域使用方向梯度直方图进行特征描述，获得 HOG 特征（下图序号 3 所示）--> 对每个小图计算特征。
4. 将获取到的 HOG 特征使用机器学习分类器（支持向量机）进行分类（下图序号 4 所示）--> 对每个小图进行分类, 是否是需要的一类。
5. 最后在图片中使用矩形框标记出被分类器认为是人的类别（下图序号 5 所示）。

![](../dataset/4_pedestrian/行人检测流程.jpg)

